# Initial value problem for ODEs

$\begin{cases} \frac{du}{dt}=f(t,u) \\
u(t=0)=u_0
\end{cases}$

Let's introduce a grid 
$\begin{cases} t_k=k\cdot \tau, k=0,1,2,3...N \\ 
u(t_k)=y_k\end{cases}$


## 1. Euler's schemes

1. Explicit Euler's scheme

$\frac{y_{k+1}-y_k}{\tau}=f(t_k,y_k)$

2. Implicit Euler's scheme

$\frac{y_{k+1}-y_k}{\tau}=f(t_{k+1},y_{k+1})$

3. Symmetrical Euler's scheme

$\frac{y_{k+1}-y_k}{\tau}=\frac{1}{2}\left(f(t_{k+1},y_{k+1})+f(t_k,y_k)\right)$

### 1.1. Approximation order

Let's define $z_n=y_n-u_n$, then for the Symmetrical Euler's scheme:

$$\frac{z_{k+1}-z_k}{\tau}=\left(-\frac{u_{k+1}-u_{k}}{\tau}+\frac{1}{2}\left(f(t_{k+1},u_{k+1})+f(t_k,u_k)\right)\right)+\frac{1}{2}\left(f(t_{k+1},u_{k+1}+z_{k+1})+f(t_k,y_k+z_k)-f(t_{k+1},u_{k+1})-f(t_k,y_k)\right)$$

$\psi_k=\frac{1}{2}\left(f(t_{k+1},u_{k+1}+z_{k+1})+f(t_k,u_k+z_k)-f(t_{k+1},u_{k+1})-f(t_k,u_k)\right)\sim z\frac{df}{du}$

$$\phi_k=-\frac{u_{k+1}-u_{k}}{\tau}+\frac{1}{2}\left(f(t_{k+1},u_{k+1})+f(t_k,u_k)\right)$$ - residual

$\phi_k=-\frac{u_k+u_k'\tau+\frac{1}{2} u_k''\tau^2+O(\tau^3)-u_k}{\tau}+\frac{1}{2}\left(f(t_{k+1},u_{k+1})+f(t_k,u_k)\right)$

$\phi_k=-u_k'-\frac{1}{2} u_k''\tau+O(\tau^2)+\frac{1}{2}\left(f(t_{k+1},u_{k+1})+f(t_k,u_k)\right)$

$f(t_{k+1},u_{k+1})=f(t_k,u_k)+\frac{df}{dt}\tau+\frac{df}{du}u'\tau=u'_k+u''_k\tau$

$\rightarrow \phi_k \sim O(\tau^2)$


### 1.2. Stability

Let's consider model equation:

$\frac{du}{dt}=\lambda u$ with $\lambda<0$

Solution is $u=u_0 e^{\lambda t}$

We know that for $t_2>t_1$ $|u(t_2)|<u|(t_1)|$ so we need $|y_{k+1}|<|y_k|$ for all $k$.

1. Explicit Euler's scheme
$y_{k+1}<y_k+\lambda \tau y_k \rightarrow |1+\lambda \tau|<1 $

1. Implicit Euler's scheme
$y_{k+1}<y_k+\lambda \tau y_{k+1} \rightarrow |\frac{1}{1-\lambda \tau}|<1 $

In [ ]:
import numpy as np
from datetime import datetime
import matplotlib.pyplot as plt
from scipy.integrate import RK45
from scipy.integrate import solve_ivp
from scipy.linalg import solve
from scipy.integrate import simps


def ExplicitEuler(f, y0, t):
    y = np.zeros([len(y0), len(t)])
    y[:, 0] = y0
    for ind in range(0, len(t)-1):
        y[:, ind+1] = y[:, ind]+(t[ind+1]-t[ind])*f(t[ind], y[:, ind])
    return y

## Consider the system with the hamiltonian $H=\frac{p^2}{2}+\frac{k^2 x^2}{2}$

Hamiltonian equations:

$\begin{cases}
\frac{dx}{dt}=p \\
\frac{dp}{dt}=-k^2x
\end{cases}$

or $\frac{d^2p}{dx^2}+k^2p=0$

$\begin{cases}
p=Asin(kt)+Bcos(kt)\\
x=\frac{B}{k}sin(kt)-\frac{A}{k}cos(kt)
\end{cases}$

Let's try to solve this system numerically.

In [ ]:
def RealSolution(t, x0, k):
    A = -x0[0]*k
    B = x0[1]
    x = -A/k*np.cos(k*t)+B/k*np.sin(k*t)
    p = A*np.sin(k*t)+B*np.cos(k*t)
    return x, p


def EnergySimple(X, k):
    H = k**2*X[0]**2/2+X[1]**2/2
    return H


def HamSimple(t, X, k):
    F = np.zeros(2)
    F[0] = X[1]
    F[1] = -k**2*X[0]
    return F


t0 = 0
tf = 10
k = 10
x0 = np.array([0, 1])
h0 = k*x0[0]**2/2+x0[1]**2/2

In [ ]:
%%timeit
N = 4000
tE = np.linspace(t0, tf, N)
tau = tE[1]-tE[0]
XE = ExplicitEuler(lambda t, X: HamSimple(t, X, k), x0, tE)

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(15, 10), sharex=True)
ax[0].plot(tE, XE[0])
ax[0].plot(tE, RealSolution(tE, x0, k)[0])
ax[0].set_ylabel('$x_{num}$')
ax[1].plot(tE, np.abs(XE[0]-RealSolution(tE, x0, k)[0]))
ax[1].set_ylabel('$|x_{num}-x_{exact}|$')

In [ ]:
plt.figure(figsize=(10, 10))
plt.plot(tE, np.abs(EnergySimple(XE, k)-h0), label='$h-h_0$')
plt.plot(tE, np.abs(XE[0]-RealSolution(tE, x0, k)[0]), label='$|x-x_{ex}|$')

plt.legend()
plt.grid()
plt.xscale('log')
plt.yscale('log')

Implicit
$$\begin{cases}x_{n+1}=x_n+\tau \,p_{n+1} \\
p_{n+1}=p_n-\tau\, k^2x_{n+1}
\end{cases}$$

In [ ]:
rsd =  # your id
np.random.seed(rsd)
kr = float(np.random.uniform(0.5, 10, 1))
x0r = np.random.uniform(-5, 5, 2)


def ImplicitEuler(t, y0, k):
    y = np.zeros([len(y0), len(t)])
    y[:, 0] = y0
    for ind in range(0, len(t)-1):
        # your code
    return y


XIE = ImplicitEuler(tE, x0r, kr)

dX = np.abs(XIE[0]-RealSolution(tE, x0r, kr)[0])
fig, ax = plt.subplots(1, 2, figsize=(15, 10), sharex=True)
ax[0].plot(tE, XIE[0])
ax[0].plot(tE, RealSolution(tE, x0r, kr)[0])
ax[0].set_ylabel('$x_{num}$')
ax[1].plot(tE, dX)
ax[1].set_ylabel('$|x_{num}-x_{real}|$')

print(simps(dX, tE))

# 2. Runge-Kutta schemes

### $\begin{cases} \frac{dU}{dt}=F(t,U) \\
U(t=0)=U_0
\end{cases}$

Basic idea - several steps on the new grid $t_{n/2}$.

Methods are explicit but allow us to construct $k$-th order approximation.

For the second order method:

1. Predictor step:

$\frac{U_{n+1/2}-U_n}{h/2}=F(t_n,U_n)$

2. Corrector step:

$\frac{U_{n+1}-U_n}{h/2}=F(t_n+\frac{h}{2},U_{n+1/2})$

Residual:

$\phi_n=-\frac{U_{n+1}-U_{n}}{h}+F(t_n+\frac{h}{2},U_n+\frac{h}{2})$

$\phi_n=-U_n'-\frac{1}{2}U_{n}''h+O(h^2)+\left(F(t_n,U_n)+\frac{h}{2}(F'+(\nabla_U F \cdot U') )\right)\sim O(h^2)$

The most common is the R-K scheme of the 4th order:

Then:

$\begin{cases}
k_1=F(t_n,U_n) \\
k_2=F(t_n+\frac{h}{2},U_n+\frac{h}{2}k_1) \\
k_3=F(t_n+\frac{h}{2},U_n+\frac{h}{2}k_2) \\
k_4=F(t_n+h,U_n+hk_3)
\end{cases}$

$U_{n+1}=U_n+\frac{h}{6}(k_1+2k_2+k_3+k_4)$

In [ ]:
% % timeit
method = 'RK23'


X = solve_ivp(fun=lambda t, Y: HamSimple(t, Y, k), t_span=(
    t0, tf), y0=x0, method=method, max_step=1e-2)

In [ ]:
% % timeit
method = 'RK45'


X = solve_ivp(fun=lambda t, Y: HamSimple(t, Y, k), t_span=(
    t0, tf), y0=x0, method=method, max_step=1e-2)

In [ ]:
% % timeit
method = 'DOP853'


X = solve_ivp(fun=lambda t, Y: HamSimple(t, Y, k), t_span=(
    t0, tf), y0=x0, method=method, max_step=1e-2)

In [ ]:
x23 = solve_ivp(fun=lambda t, Y: HamSimple(t, Y, k), t_span=(
    t0, tf), y0=x0, method='RK23', max_step=1e-2)
x45 = solve_ivp(fun=lambda t, Y: HamSimple(t, Y, k), t_span=(
    t0, tf), y0=x0, method='RK45', max_step=1e-2)
x853 = solve_ivp(fun=lambda t, Y: HamSimple(t, Y, k), t_span=(
    t0, tf), y0=x0, method='DOP853', max_step=1e-2)



h23 = EnergySimple(x23.y, k)
h45 = EnergySimple(x45.y, k)
h853 = EnergySimple(x853.y, k)

In [ ]:
fig, ax = plt.subplots(2,3,figsize=(15,10),sharex=True)
ax[0,0].plot(x23.t,x23.y[0],label='RK23')
ax[0,0].plot(x23.t,RealSolution(x23.t,x0,k)[0])

ax[0,1].plot(x45.t,x45.y[0],label='RK45')
ax[0,1].plot(x45.t,RealSolution(x45.t,x0,k)[0])

ax[0,2].plot(x853.t,x853.y[0],label='DOP853')
ax[0,2].plot(x853.t,RealSolution(x853.t,x0,k)[0])

ax[0,0].set_ylabel('$x_{num}$')

ax[1,0].plot(x23.t,np.abs(x23.y[0]-RealSolution(x23.t,x0,k)[0]))
ax[1,1].plot(x45.t,np.abs(x45.y[0]-RealSolution(x45.t,x0,k)[0]))
ax[1,2].plot(x853.t,np.abs(x853.y[0]-RealSolution(x853.t,x0,k)[0]))
ax[1,0].set_ylabel('$|x_{num}-x_{real}|$')
for axis in ax[0,:]:
    axis.legend()

In [ ]:
fig, ax = plt.subplots(2,3,figsize=(15,10))
ax[0,0].plot(x23.y[0],x23.y[1])
ax[0,1].plot(x45.y[0],x45.y[1])
ax[0,2].plot(x853.y[0],x853.y[1])

ax[1,0].plot(x23.t,np.abs(h23-h0))
ax[1,1].plot(x45.t,np.abs(h45-h0))
ax[1,2].plot(x853.t,np.abs(h853-h0))